# 3 Beispielabfragen

## 3.0 Eingabe personalisierter Parameter

### 3.0.1 Datenbankparameter

In [ ]:
DATABASE_NAME = 'YOUR_DATABASE_NAME'
DATABASE_USER = 'YOUR_DATABASE_USER'
DATABASE_PASSWORD = 'YOUR_DATABASE_PASSWORD'

### 3.0.2 Parameter für Abfrage 3.2

#### 3.0.2.1 Zu betrachtende Länder

In [ ]:
countries = ['DE', 'IT', 'GB', 'FR', 'NL'] #Liste beliebig erweiterbar

#### 3.0.2.2 Zu betrachtendes Item

In [ ]:
item_1 = 82 #Gewinn/Verlust

### 3.0.3 Parameter für Abfrage 3.3

#### Zu betrachtende Bank

In [ ]:
bank = 15 #Landesbank Baden-Württemberg

## 3.1 Import relevanter Funktionen und Herstellen der Vebindung zur Datenbank

In [ ]:
import pandas as pd
from pandas.plotting import parallel_coordinates
from src.core.Connection import getConnection, getRootConnection

connection = getConnection(DATABASE_NAME, DATABASE_USER, DATABASE_PASSWORD)
cursor = connection.cursor()

## 3.2 Gesamtgewinn /-verlust nach Periode nach Land

### 3.2.1 Daten aus Datenbank laden

In [ ]:
query = f"""SELECT DISTINCT
                  b.Country_Code as Country_Code,
                  o.Period as Period,
                  o.Amount as Amount,
                  o.GlobalLEI_Code as BankID
           FROM others as o, bank b
           WHERE o.ItemID = '{item_1}' AND
                 o.GlobalLEI_Code = b.GlobalLEI_Code
           ORDER BY o.Period""" 

data = cursor.execute(query)

### 3.2.2 Daten in Dataframe speichern (Period als einzelne als Spalte)

In [ ]:
df = pd.DataFrame(cursor.fetchall())
df.columns = cursor.column_names
df = df.sort_values(by = ['BankID', 'Period'])
df.style.hide_index()

### 3.2.3 Dataframe so verändern, dass für jede Periode eine Spalte entsteht

In [ ]:
df_copy = df.copy()

df = df.drop_duplicates(subset=['BankID']).copy()
df.drop(['Period', 'Amount'], axis=1, inplace=True)
    
for index, row in df_copy.iterrows():
    period = row['Period']
    index_of_bank = df[df['BankID']==row['BankID']].index.values
    df.loc[index_of_bank, period] = row['Amount']

for col in df.columns:
    if (col != 'Country_Code'):
        df[col]=pd.to_numeric(df[col])
        
df.style.hide_index()

### 3.2.4 Daten nach Ländern gruppieren

In [ ]:
df_grouped = df.groupby(df['Country_Code']).mean().reset_index()
df_grouped.drop('BankID', axis=1, inplace=True)

df_grouped.style.hide_index()

### 3.2.5 Daten visualisieren

In [ ]:
f = df_grouped[df_grouped['Country_Code'].isin(countries)]

parallel_coordinates(f, 'Country_Code')

## 3.3 Zinseinkommen für eine einzelne Bank

### 3.3.1 Daten aus Datenbank laden

In [ ]:
query = f"""SELECT DISTINCT
                  o.Period as Period,
                  o.Amount as Amount,
                  o.ItemID as ItemID
           FROM others as o
           WHERE (o.ItemID = '48' OR
                 o.ItemID = '49' OR
                 o.ItemID = '50') AND
                 o.GlobalLEI_Code = '{bank}'
           ORDER BY o.Period, o.ItemID""" 

data = cursor.execute(query)

### 3.3.2 Daten in Dataframe speichern (Item als einzelne als Spalte)

In [ ]:
df = pd.DataFrame(cursor.fetchall())
df.columns = cursor.column_names
df.style.hide_index()

### 3.3.3 Dataframe so verändern, dass für jedes Item eine Spalte entsteht

In [ ]:
df_copy = df.copy()

df = df.drop_duplicates(subset=['Period']).copy()
df.drop(['ItemID', 'Amount'], axis=1, inplace=True)
    
for index, row in df_copy.iterrows():
    item = row['ItemID']
    index_of_period = df[df['Period']==row['Period']].index.values
    df.loc[index_of_period, item] = row['Amount']

for col in df.columns:
    if (col != 'ItemID'):
        df[col]=pd.to_numeric(df[col])
        
df = df.set_index('Period')
df

### 3.3.4 Daten visualisieren

In [ ]:
# Label für Legende aus Datenbank holen
query = 'SELECT Label FROM itemmeta WHERE GlobalID = 48 OR GlobalID = 49 OR GlobalID = 50'
data = cursor.execute(query)
labels = pd.DataFrame(cursor.fetchall())
labels.columns = cursor.column_names

ax = df.plot.bar(y=48, rot=0)
ax.legend([labels.loc[0, 'Label']])

ax_stacked = df.loc[:, df.columns.difference([48])].plot.bar(stacked=True, rot=0)
ax_stacked.legend(labels.loc[1:2, 'Label'].tolist())